In [2]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
#!pip install gmaps

In [1]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faanui,-16.48,-151.75,79.93,76,3,24.43,PF,1590201369
1,1,dariba,24.95,74.13,97.57,16,0,16.17,IN,1590201369
2,2,jamestown,42.10,-79.24,62.01,100,90,3.36,US,1590201233
3,3,camacha,33.08,-16.33,66.20,82,75,10.29,PT,1590201369
4,4,upernavik,72.79,-56.15,28.22,97,8,9.31,GL,1590201369
5,5,bathsheba,13.22,-59.52,80.60,78,40,14.99,BB,1590201369
6,6,kodiak,57.79,-152.41,50.00,71,75,9.17,US,1590201370
7,7,hilo,19.73,-155.09,80.60,57,75,8.05,US,1590201334
8,8,port lincoln,-34.73,135.87,58.53,67,92,15.75,AU,1590201370
9,9,mataura,-46.19,168.86,54.19,73,87,1.83,NZ,1590201370


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=gkey)

In [7]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,baykit,61.67,96.37,78.60,25,0,3.22,RU,1590201371
150,150,hami,42.80,93.45,73.58,28,0,6.51,CN,1590201380
160,160,sao felix do xingu,-6.64,-51.99,73.00,94,0,2.15,BR,1590201380
184,184,urumqi,43.80,87.60,75.20,25,0,6.71,CN,1590201383
214,214,caravelas,-17.71,-39.25,72.97,93,0,2.77,BR,1590201386
263,263,sao filipe,14.90,-24.50,73.87,72,0,7.00,CV,1590201393
282,282,teya,60.38,92.63,78.12,20,0,4.94,RU,1590201394
287,287,jalpan,21.23,-99.48,77.00,70,0,1.90,MX,1590201395
325,325,petatlan,17.52,-101.27,74.37,87,0,3.31,MX,1590201398
326,326,dakar,14.69,-17.44,71.60,88,0,2.24,SN,1590201258


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
25,baykit,RU,61.67,96.37,
150,hami,CN,42.80,93.45,
160,sao felix do xingu,BR,-6.64,-51.99,
184,urumqi,CN,43.80,87.60,
214,caravelas,BR,-17.71,-39.25,
263,sao filipe,CV,14.90,-24.50,
282,teya,RU,60.38,92.63,
287,jalpan,MX,21.23,-99.48,
325,petatlan,MX,17.52,-101.27,
326,dakar,SN,14.69,-17.44,


In [10]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
25,baykit,RU,61.67,96.37,Gostinitsa
150,hami,CN,42.80,93.45,Laoyutang Resort
160,sao felix do xingu,BR,-6.64,-51.99,Apart Hotel Rio Xingu
184,urumqi,CN,43.80,87.60,Hua Ling Grand Hotel Urumqi
214,caravelas,BR,-17.71,-39.25,Pousada dos Navegantes
263,sao filipe,CV,14.90,-24.50,Tortuga B&B
282,teya,RU,60.38,92.63,
287,jalpan,MX,21.23,-99.48,Hotel Mision Jalpan
325,petatlan,MX,17.52,-101.27,Hotel California Petatlán
326,dakar,SN,14.69,-17.44,Novotel Dakar


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))